In [1]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import traceback
import random
import openpyxl
import os




def get_driver():
    # read proxy list from proxy_list.txt and save it in a list. Also remove line that is empty
    with open("proxy_list.txt", "r") as file:
        proxy_list = file.read().splitlines()
        proxy_list = [x for x in proxy_list if x]



    chromeOptions = webdriver.ChromeOptions()

    # Headless is faster. If headless is False then it opens a browser and you can see action of web driver. You can try making it False
    chromeOptions.headless = False
    chromeOptions.add_argument("--log-level=3")

    # use rotating proxy. Configuration geo.iproyal allows for rotating proxy servers automatically. If you do not have proxy then increase sleep time in random_sleep function
    '''
    proxy = random.choice(proxy_list)
    print(f"Using proxy: {proxy}")
    chromeOptions.add_argument('--proxy-server={}'.format(proxy))
    '''
    # installs chrome driver automatically if not present
    s = Service(ChromeDriverManager().install())

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=chromeOptions
    )
    return driver

def remove_matching_entity_ids(scraped_entity_ids, all_entity_ids):
    # remove matching entity ids
    for entity_id in scraped_entity_ids:
        all_entity_ids.remove(entity_id.replace("-", ""))
    return all_entity_ids[:500]

def random_sleep(min_sec=5, max_sec=12):
    # avoiding ban
    time.sleep(random.randint(min_sec, max_sec))

def entity_id_list():
    result = []
    for i in range(965):  # 000 to 964
        for j in range(1000):  # 000 to 999
            for k in range(1, 1000):  # 000 to 999
                current_string = f"{i:03d}{j:03d}{k:03d}"
                result.append(current_string)
                if current_string == "000964437":
                    return result

In [2]:
filename = 'result.xlsx'

entity_id_list = entity_id_list()
records = []
if os.path.exists(filename):
    df = pd.read_excel(filename)
    records = df.to_dict(orient='records')
    scraped_entity_ids = df["Entity ID Number"].tolist()
    # remove matching entity ids
    entity_id_list = remove_matching_entity_ids(scraped_entity_ids, entity_id_list)

In [3]:
def detailed_scrap(entity_id_list):
    # we have to visit each new link to scrap detaily so it takes time.
    url = "https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp="
    driver = get_driver()
    count = 0


    for entity_id in entity_id_list:
        try:
            '''
            count += 1
            if count % 10 == 0:
                driver.close()
                driver = get_driver()
            '''
            driver.get(url+entity_id)
            
            if driver.find_elements(By.ID, "main-frame-error"):
                driver.close()
                print("Proxy did not work so quit it. Using another proxy")
                driver = get_driver()
                driver.get(url+entity_id)
                
            random_sleep()
            info_dict = {}
            try:
                if driver.find_element(By.ID, "block-sos-content").text == 'No matches found.':
                    info_dict['Entity ID Number'] = entity_id[:3] + '-' + entity_id[3:6] + '-' + entity_id[6:9]
                    info_dict['Status'] = 'No matches found.'
                    records.append(info_dict)

                    continue
                infos = [info.find_elements(By.TAG_NAME, "td") for info in driver.find_element(By.TAG_NAME, "tbody").find_elements(By.TAG_NAME, "tr")]
            except:
                continue
            
            
            for info in infos:
                try:
                    key = info[0].text
                    value = info[1].text
                    info_dict[key] = value
                except IndexError:
                    pass
            print("hello")
            
            info_dict['Entity Name'] = driver.find_element(By.TAG_NAME, 'td').text # first td is entity name

            records.append(info_dict)
            if len(records) % 5 == 0:
                print("Number of records scraped:", len(records))
                pd.DataFrame.from_records(records).to_excel(filename, index=False)
        
        except:
            driver.save_screenshot('screenshot.png')
            print(driver.current_url)
            print(traceback.format_exc())
            driver.close()
            driver = get_driver()
        
    if len(records) > 0:
        pd.DataFrame.from_records(records).to_excel(filename, index=False)

    driver.quit()

In [4]:

# if you want to use multithreading use this. But be careful. Simultanus website visit from same IP can lead to ban. So I have avoided it after trying
max_workers = 2
each_list_len = int(len(entity_id_list)/max_workers)+1
list_of_list = []
for i in range(max_workers-1):
    list_of_list.append(entity_id_list[each_list_len * i : each_list_len* (i + 1)])
list_of_list.append(entity_id_list[each_list_len * (i + 1) :])

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    executor.map(detailed_scrap, list_of_list)


Using proxy: 45.251.61.113:6831
Using proxy: 137.59.4.152:6021


In [ ]:
import pandas as pd
filename = 'result.xlsx'
df = pd.read_excel(filename).sort_values(by='Entity ID Number')
df.to_excel('result_final.xlsx', index=False)